# 商品期货跨品种策略

## 摘要

# 理论模型

## 产业链套利

产业链套利是产业链内部相关品种间的套利模式，主要有：
* 炼钢套利，涉及品种：螺纹钢，铁矿石，焦炭，详见: [炼钢套利](.\炼钢套利.ipynb)
* 炼焦套利，涉及品种：焦煤，焦炭，详见：[炼焦套利](.\炼焦套利.ipynb)
* 甲醇制PP套利，涉及品种：甲醇，聚丙烯，详见：[甲醇制PP套利](.\甲醇制PP套利.ipynb)
* 产蛋套利，涉及品种：鸡蛋，豆粕，玉米，详见：[产蛋套利](.\产蛋套利.ipynb)
* 大豆提油套利，涉及品种：大豆，豆粕，豆油，详见：[大豆提油套利](.\大豆提油套利.ipynb)

## 替代品种套利

替代品种套利是指在相同大类中有相似功能的品种之间的套利模式，由于功能的相互替代性期货价格呈现一定相关性，当价差偏离较大的时候可以入场进行套利。


## 内外盘套利

目前国内商品中对应有外盘的品种为：大豆，豆粕，豆油，玉米，黄金，白银，白糖，棉花，铜，铝，铅，锌，镍。

内外盘套利主要是通过统计套利或者计算内外盘品种间的无套利成本区间把握套利的机会。

## 对冲策略

该类策略交易的对象是市场上全部活跃的合约，做多强势的，做空弱势的，或者进一步要求在各个期货大类内部实现多空平衡，比如黑色系、有色金属、能源化工、农产品、油脂油料、贵金属等。

In [ ]:
# 所有期货的展期收益率
TargetDT = dt.datetime.combine(dt.date.today(), dt.time(0))-dt.timedelta(1)# 设置目标日期

WDB.connect()

# 获取所有的主力合约和次主力合约
FutureCodes = WDB.getFutureCode(exchange=["SHFE", "INE", "DCE", "CZCE"], date=TargetDT, is_current=True)
FutureIDs = WDB.getFutureID(future_code=FutureCodes, date=TargetDT, is_current=False, contract_type="连续合约")
FutureIDs = pd.Series(FutureIDs)
FutureIDs = FutureIDs[~FutureIDs.str.findall("\d+").apply(bool)]
FutureInfo = pd.DataFrame(FutureIDs[~FutureIDs.str.findall("_S").apply(bool)].values, columns=["主力合约"])
FutureInfo["次主力合约"] = FutureIDs[FutureIDs.str.findall("_S").apply(bool)].values
FutureInfo.index = FutureInfo["主力合约"].apply(lambda x: x.split(".")[0]).values

# 获取主力和次主力合约在目标日的月合约 ID
FT = WDB.getTable("中国期货连续(主力)合约和月合约映射表")
FutureInfo["主力月合约"] = FT.readData(factor_names=["映射月合约Wind代码"], ids=FutureInfo["主力合约"].tolist(), dts=[TargetDT]).iloc[0, 0].values
FutureInfo["次主力月合约"] = FT.readData(factor_names=["映射月合约Wind代码"], ids=FutureInfo["次主力合约"].tolist(), dts=[TargetDT]).iloc[0, 0].values

# 去掉缺失值
FutureInfo.dropna(how="any", inplace=True)

# 获取主力和次主力月合约的到期日
FT = WDB.getTable("中国期货基本资料")
FutureInfo["主力月合约最后交易日"] = FT.readData(factor_names=["最后交易日期"], ids=FutureInfo["主力月合约"].tolist(), dts=[TargetDT]).iloc[0, 0].values
FutureInfo["次主力月合约最后交易日"] = FT.readData(factor_names=["最后交易日期"], ids=FutureInfo["次主力月合约"].tolist(), dts=[TargetDT]).iloc[0, 0].values

# 获取主力和次主力月合约在目标日的价格
FT = WDB.getTable("中国商品期货日行情")
FutureInfo["主力合约结算价"] = FT.readData(factor_names=["结算价(元)"], ids=FutureInfo["主力合约"].tolist(), dts=[TargetDT]).iloc[0,0].values
FutureInfo["次主力合约结算价"] = FT.readData(factor_names=["结算价(元)"], ids=FutureInfo["次主力合约"].tolist(), dts=[TargetDT]).iloc[0,0].values

WDB.disconnect()

# 计算展期收益率
TimeDelta = FutureInfo["次主力月合约最后交易日"].apply(lambda x: dt.datetime.strptime(x, "%Y%m%d")) - FutureInfo["主力月合约最后交易日"].apply(lambda x: dt.datetime.strptime(x, "%Y%m%d"))
TimeDelta = TimeDelta.apply(lambda x: x.days / 365)
FutureInfo["展期收益率"] = (np.log(FutureInfo["主力合约结算价"]) - np.log(FutureInfo["次主力合约结算价"])) / TimeDelta

display(FutureInfo)